# Advanced Data Science &amp; Python for Finance  <br><br> Portfolio Backtesting 

-----

FIN580-59305

Jose Luis Rodriguez

Director of Margolis Market Information Lab at University of Illinois at Urbana-Champaign.

* linkedin.com/in/jlroo
* github.com/jlroo

-----

* [Intrinio API](#intrinio)
* [Common Financial Analyses](#commonanalyses)
* [Building A Trading Strategy](#tradingstrategy)
* [Backtesting with Pandas and Matplotlib](#backtesting)
* [Backtrader](#backtrader)
    

## Packages and Settings

First make sure that the API credentials are stored in a secure file to minimize exposure. We will use the package ``configparses`` to reach the credentials.

**Configuration**

In [ ]:
import backtrader as bt
import matplotlib.pyplot as plt
%matplotlib inline

**Scientific Analysis**

In [ ]:
import pandas as pd

<a id='backtrader'></a>

# Backtrader 

**Strategy Overview**

Given current market conditions and increase in volatility we may be able to take advantage of a simple moving average pair with rsi mean reversion on a portfolio of SP1500 stocks. 
 
**Strategy**

SP1500 Communication Equipment stocks will rank by 3-day moving average. From that rank we will take the lower 10 stocks and rank them by their 10-day relative strength indicator (RSI) to determine what stocks may be oversold.

Top two with highest RSI will be candidate for a buy and sell previous positions rebalance every Monday.

**Trade Management and Position Sizing**

* Position size buy/sell 100 shares at a time without 
* Backtrader will reject order if not enough funds

## Data Preparation: Backtrader Data format

In [ ]:
oil_df = pd.read_csv("../data/oil_df-dec.csv", index_col= 0)

In [ ]:
oil_df.columns

In [ ]:
oil_df['secid'].unique()

**Create a new dataframe for backtrader**

In [ ]:
bt_data = oil_df[['adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume', 'secid']]

**Change the name of the columns to work with backtrader**

In [ ]:
bt_data = bt_data.rename(columns={'adj_open':'open', 'adj_high':'high', 'adj_low':'low',
                                  'adj_close':'close', 'adj_volume':'volume', 'secid':'name'})
bt_data.index.name = None

**Save the complete prepared dataset with industry constituents stock prices**

In [ ]:
bt_data.to_csv("../data/bt_oil.csv")

**We also need to save each individual stock price as separate file**

In [ ]:
constituents = bt_data['name'].unique()

In [ ]:
# Each stock price is saved to a folder inside the data folder (btdata)
for stock in constituents:
     bt_data[bt_data['name'] == stock].to_csv("../data/btdata/" + stock + ".csv")

**NOTE: Only run the code below once to save all the individual stock prices to teh data folder ``../data/btdata``**

## Creating a Backtrader Strategy 

**Read the constituents stock data into backtrader**

In [ ]:
bt_data = pd.read_csv("../data/bt_oil.csv",index_col = 0 )
constituents = bt_data.name.unique()
len(constituents)

**Strategy Class**

In [ ]:
class MomentumStrategy(bt.Strategy):
    params = dict( 
        num_universe = 32, # Number of Industry Constituents 
        num_positions = 2, # Set the number of position to hold at any given time
        when = bt.timer.SESSION_START,
        weekdays = [5],
        weekcarry = True,
        rsi_period = 8, # Relative Strength Index Periods
        sma_period = 18 # Moving Average Periods
    )
    
    def __init__(self):
        self.inds = {}
        self.rsi = {}
        
        self.securities = self.datas[1:]
        for s in self.securities:
            self.inds[s] = {}
            self.inds[s]['sma'] = bt.ind.SMA(s, period = self.p.sma_period)
            self.inds[s]['sma'].plotinfo.plot = False
            self.inds[s]['rsi'] = bt.ind.RSI(s, period = self.p.rsi_period)
            self.inds[s]['rsi'].plotinfo.plot = False
        
        self.add_timer(
            when = self.p.when,
            weekdays = self.p.weekdays,
            weekcarry = self.p.weekcarry
        )
        
    def notify_timer(self, timer, when, *args, **kwargs):
            self.rebalance()
        
    def notify_trade(self, trade):
        if trade.size == 0:
            print("DATE:", trade.data.datetime.date(ago=0),
                  " TICKER:", trade.data.p.name, 
                  "\tPROFIT:", trade.pnlcomm)

    def rebalance(self):
        
        rankings = list(self.securities)
        
        rankings.sort(
            key = lambda s: self.inds[s]['sma'][0],
            reverse = False
        )
        
        rankings = rankings[:self.p.num_universe]
        
        rankings.sort(
            key = lambda s: self.inds[s]['rsi'][0],
            reverse = True
        )

        # position size short
        pos_size = -1 / self.p.num_positions 

        # Sell when ranking
        for i, d in enumerate(rankings):
            if self.getposition(d).size:
                if i > self.p.num_positions:
                    self.close(d)

        # Buy and rebalance stocks with remaining cash
        for i, d in enumerate(rankings[:self.p.num_positions]):
            self.order_target_percent(d, target = pos_size)
            

**Cerebro Parameters**

In [ ]:
starcash = 100000

In [ ]:
cerebro = bt.Cerebro()
cerebro.broker.setcash(starcash)
cerebro.broker.setcommission(commission=0.0)

**Feed data to backtrader**

In [ ]:
first_stock = True
for stock in constituents:
    # Load the each stock price data from the btdata folder 
    filename = "../data/btdata/" + stock + ".csv"
    data = bt.feeds.GenericCSVData(
        dataname = filename,
        dtformat = ('%Y-%m-%d'),
        datetime = 0,
        high = 2,
        low = 3,
        open = 1,
        close = 4,
        volume = 5,
        openinterest = -1,
        name = stock)
    if first_stock:
        data0 = data
        data.plotinfo.sameaxis = False
        data.plotinfo.plotylimited = True
        first_stock = False
    else:
        data.plotinfo.plotmaster = data0
        data.plotinfo.subplot = False
        data.plotinfo.sameaxis = False
        data.plotinfo.plotylimited = True
    cerebro.adddata(data, name = stock)

In [ ]:
cerebro.addstrategy(MomentumStrategy)

In [ ]:
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)

In [ ]:
# When run section you should see trades excuted
# If there are no trades excuted modifed the RSI and/or the Moving Average periods.

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
result = cerebro.run()
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
dd = result[0].analyzers.drawdown.get_analysis()['max']['drawdown']
cagr = result[0].analyzers.returns.get_analysis()['rnorm100']
sharpe = result[0].analyzers.sharperatio.get_analysis()['sharperatio']

print(f"Max Drawdown: {dd:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")

In [ ]:
plt.rcParams['figure.figsize'] = [38, 32]
cerebro.plot(volume=False)